In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("clean_train_reviewsbg.csv")

In [3]:
train.head(1)

,id,sentiment,review,cleanReview
0,"""5814_8""",1,"""With all this stuff going down at the moment ...",stuff going moment mj started listening music ...


In [4]:
len(train)

25000

In [5]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(train['cleanReview'], train['sentiment'], test_size = 0.33, random_state = 42)

In [6]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [7]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [8]:
tokenizer_obj = Tokenizer()
total_reviews = train['cleanReview']
tokenizer_obj.fit_on_texts(total_reviews)

In [9]:
max_length = max([len(s.split()) for s in total_reviews])

In [11]:
vocab_size = len(tokenizer_obj.word_index) + 1

In [12]:
Xtraintok = tokenizer_obj.texts_to_sequences(train_features)
Xtesttok = tokenizer_obj.texts_to_sequences(test_features)

In [13]:
X_train_pad = pad_sequences(Xtraintok, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(Xtesttok, maxlen=max_length, padding='post')
print(len(X_test_pad))
print(len(test_labels))

8250
8250


In [14]:
from keras.models import Sequential 

In [15]:
from keras.layers import Embedding, Dense, LSTM, GRU

In [16]:
from keras.layers.embeddings import Embedding
from keras import optimizers
### word embedding mapping a word a learn repesnentaion: an array #numeric space is not meaningful us for people 

In [17]:
EMBEDDING_DIM = 100  ##negative or positive a word is 

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
## entire review 10 words by 10 x 100 

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1416, 100)         7406600   
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 7,419,401
Trainable params: 7,419,401
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train_pad, train_labels, batch_size=100, epochs=3, validation_data=(X_test_pad, test_labels), verbose=2)

Train on 16750 samples, validate on 8250 samples
Epoch 1/3
 - 394s - loss: 0.6934 - val_loss: 0.6938
Epoch 2/3
 - 412s - loss: 0.6935 - val_loss: 0.6934
Epoch 3/3
 - 414s - loss: 0.6933 - val_loss: 0.6935


In [24]:
sen_pred_train = model.predict(X_train_pad)
sen_pred_test = model.predict(X_test_pad)

In [29]:
print(sen_pred_test)

[[0.4892042 ]
 [0.4892042 ]
 [0.4892042 ]
 ...
 [0.4892042 ]
 [0.48920423]
 [0.48920423]]


In [28]:
print(test_labels)

6868     0
24016    1
9668     0
13640    1
14018    0
7488     1
5804     1
12909    0
3386     0
9567     0
21423    0
3503     0
6657     0
19192    1
2518     1
13361    1
21135    0
21080    1
6492     0
6891     0
7752     1
8614     0
6649     1
2227     1
12060    1
19122    0
9622     1
12279    1
16447    0
423      1
        ..
47       1
8956     0
13267    1
24606    0
12133    1
13093    0
17555    1
12509    1
3289     1
23121    0
4755     1
16426    0
18307    1
16178    0
18516    0
23198    1
11472    0
15436    1
9048     0
1871     0
23068    1
22818    0
418      0
1213     1
16115    0
10802    0
7160     0
2499     0
12899    0
18149    1
Name: sentiment, Length: 8250, dtype: int64


In [30]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [31]:
review_lines = list()

In [32]:
lines = train['review'].values.tolist()

In [33]:
for line in lines:
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [34]:
len(review_lines)

25000

In [35]:
import gensim

In [36]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)


C:\Users\ngmat\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [38]:
model.get_vector('sad').shape

(300,)

In [39]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [40]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4 # which review is the longest number of words
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [41]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = model.get_vector(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [42]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.07568359  0.03369141 -0.06494141 ...  0.01202393  0.13574219
  -0.09130859]
 [-0.04931641  0.24121094 -0.03222656 ... -0.04077148  0.00130463
  -0.17871094]
 ...
 [ 0.12353516  0.03198242  0.15039062 ... -0.21289062 -0.00643921
  -0.08007812]
 [-0.13964844 -0.03466797 -0.05371094 ... -0.11816406 -0.00245667
  -0.07226562]
 [-0.19433594 -0.06152344  0.00915527 ... -0.08007812 -0.08740234
  -0.00427246]]


In [43]:
model.most_similar(positive=['sad'])

[('saddening', 0.7273085713386536),
 ('Sad', 0.6610826253890991),
 ('saddened', 0.6604381799697876),
 ('heartbreaking', 0.6573508381843567),
 ('disheartening', 0.6507317423820496),
 ('saddens_me', 0.6407119035720825),
 ('distressing', 0.6399092674255371),
 ('saddest', 0.6345508694648743),
 ('unfortunate', 0.6272087097167969),
 ('sorry', 0.619404673576355)]

In [45]:
EMBEDDING_DIM = 300  ##negative or positive a word is 

In [46]:
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length, weights=[embedding_matrix]))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [47]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [48]:
model.fit(padded_docs, labels, epochs=10)

C:\Users\ngmat\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
10/10 [==============================] - 1s 89ms/step - loss: 0.6881
Epoch 2/10
10/10 [==============================] - 0s 601us/step - loss: 0.6724
Epoch 3/10
10/10 [==============================] - 0s 499us/step - loss: 0.6807
Epoch 4/10
10/10 [==============================] - 0s 399us/step - loss: 0.6628
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 0.6510
Epoch 6/10
10/10 [==============================] - 0s 499us/step - loss: 0.6542
Epoch 7/10
10/10 [==============================] - 0s 497us/step - loss: 0.6261
Epoch 8/10
10/10 [==============================] - 0s 499us/step - loss: 0.6235
Epoch 9/10
10/10 [==============================] - 0s 399us/step - loss: 0.6149
Epoch 10/10
10/10 [==============================] - 0s 896us/step - loss: 0.6045
